In [2]:
!pip install --upgrade --quiet pinecone-client pinecone-text pinecone-notebooks


In [7]:
import os 
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("PINECONE_API_KEY")
os.environ["HF_API_KEY"] = os.getenv("HF_API_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")



In [8]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [9]:
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain-pinecone"
### Initialize the pinecone client
pc=Pinecone(api_key=api_key)

## Create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=1024,# dimension of dense vector 
        metric='dotproduct', # sparse value suppoted only for dot product
        spec=ServerlessSpec(cloud="aws",region="us-east-1"),

    )


In [10]:
index=pc.Index(index_name)
index

In [11]:
import os
from langchain_huggingface import HuggingFaceEmbeddings

embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings



c:\Users\Arun\AI tamil\NLP\Langchain\venv\lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Arun\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\Arun\AI tamil\NLP\Langchain\venv\lib\site-packages\transformers\tokenization

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [13]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()#tfidf
bm25_encoder

In [14]:
sentence=[
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
]

In [15]:
#TFIDF values on these sentence
bm25_encoder.fit(sentence)

# sotre value in json file
bm25_encoder.dump("bm25_values.json")

# Load to your BM25Encoder object
bm25_encoder=BM25Encoder().load("bm25_values.json")

100%|██████████| 3/3 [00:00<00:00, 100.01it/s]


In [16]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)


In [17]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x0000015BFD7E3640>, index=<pinecone.data.index.Index object at 0x0000015BE0783670>)

In [18]:
retriever.add_texts(
    [
        "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans"
    ]
)

100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


In [19]:
retriever.invoke("What city did i visit last")

[Document(page_content='In 2021, I visited New Orleans'),
 Document(page_content='In 2022, I visited New York'),
 Document(page_content='In 2023, I visited Paris')]